# Data Preparation

In [268]:
import pandas
import matplotlib.pyplot as plt
from collections import Counter
import string
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses

wf = pandas.read_csv("/home/yash/Code/PresidentialSpeechClassification/EMPOLITICON.csv")
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,16/07/2021,Vladimir Putin,Meeting of APEC Economic Leaders,"Madam Chair,\n\nColleagues,\n\nFirst of all, ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,OPTIMISM,DEVELOPMENT
1,Russia,2021-09-05 00:00:00,Vladimir Putin,Victory Parade on Red Square,"Citizens of Russia,\n\nDear veterans,\n\nComra...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,NATIONALISM
2,Russia,2021-08-04 00:00:00,Vladimir Putin,Meeting on the results of implementing Preside...,"Good afternoon, colleagues.\n\nLet’s start.\n\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
3,Russia,21-11-2020,Vladimir Putin,G20 Summit,"Colleagues,\n\nThe scope of problems humanity ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
4,Russia,20-11-2020,Vladimir Putin,Address to participants in Nuremberg Lessons f...,"Colleagues, friends,\n\nFirst of all, I would ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,EXTREMISM
...,...,...,...,...,...,...,...,...,...,...
2005,USA,05/02/2013,Barack Obama,\nRemarks by the President,"Good afternoon, everybody. \n\nI wanted to sa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,DEVELOPMENT
2006,USA,04/02/2013,Barack Obama,\nRemarks by the President on Preventing Gun V...,"Hello, everybody. Please have a seat. Have a...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,OTHERS
2007,USA,02/02/2013,Barack Obama,\nWeekly Address: A Balanced Approach to Growi...,"Hi, everybody. \n\nIn the coming weeks, we fa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,DEVELOPMENT
2008,USA,28/01/2013,Barack Obama,\nRemarks by the President Before Meeting with...,"Well, Vice President Biden and I just want to...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,OTHERS


In [269]:
# shuffing the data
wf = wf.sample(frac=1).reset_index(drop=True)
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,27/03/2015,Barack Obama,\nRemarks by the President in Meeting with Mem...,I’m happy to have an opportunity to meet with...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,DEVELOPMENT
1,China,2020-11-10 00:00:00,H.E. Xi Jinping,Carrying Forward the Shanghai Spirit and Deepe...,"Your Excellency President Vladimir Putin,\nDea...",President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,NEUTRAL,INTERNATIONAL AFFAIRS
2,USA,2017-04-18 00:00:00,Donald Trump,"Remarks by President Trump on Buy American, Hi...",THE PRESIDENT: Thank you very much. (Applaus...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,OPTIMISM,DEVELOPMENT
3,Russia,2017-09-05 00:00:00,Vladimir Putin,Military parade on Red Square,"Citizens of Russia, veterans, comrade soldiers...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,NATIONALISM
4,Russia,17-09-2003,Vladimir Putin,Speech at a Meeting on Issues of the Russian M...,"Good afternoon, dear colleagues. First of all,...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,INTERNATIONAL AFFAIRS
...,...,...,...,...,...,...,...,...,...,...
2005,United Kingdom,24 July 2019,Boris Johnson,Boris Johnson's first speech as Prime Minister...,Good afternoon I have just been to see Her Maj...,Prime Minister,Boris Johnson,https://www.gov.uk/government/speeches/boris-j...,OPTIMISM,DEVELOPMENT
2006,Russia,2009-09-02 00:00:00,Dmitry Medvedev,Opening Remarks at Meeting on Economic Issues,"Colleagues,\n\nToday we are meeting to discuss...",President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,UPSET,DEVELOPMENT
2007,China,2018-07-20 00:00:00,H.E. Xi Jinping,Full text of Chinese President Xi's signed art...,I will pay a state visit to the United Arab Em...,President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,NEUTRAL,INTERNATIONAL AFFAIRS
2008,Russia,2007-09-05 00:00:00,Vladimir Putin,Speech at the Military Parade Celebrating the ...,"Citizens of Russia,\n\nVeterans of the Great P...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,NATIONALISM


In [270]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Emotion']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [271]:
print(counter.most_common())

[('NEUTRAL', 589), ('OPTIMISM', 583), ('JOY', 525), ('UPSET', 313)]


Setting labels for a multi-classification problem

In [272]:
#labeling the emotion column such that the most common emotion is 0, the second most common is 1 and so on
for i in range(len(wf)):
    if wf['Emotion'][i] == counter.most_common()[0][0]:
        wf['Emotion'][i] = 0
    elif wf['Emotion'][i] == counter.most_common()[1][0]:
        wf['Emotion'][i] = 1
    elif wf['Emotion'][i] == counter.most_common()[2][0]:
        wf['Emotion'][i] = 2
    elif wf['Emotion'][i] == counter.most_common()[3][0]:
        wf['Emotion'][i] = 3

#these labels will be used to train the model

/tmp/ipykernel_1108/1047653752.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Emotion'][i] = 1
/tmp/ipykernel_1108/1047653752.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values t

In [273]:
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,27/03/2015,Barack Obama,\nRemarks by the President in Meeting with Mem...,I’m happy to have an opportunity to meet with...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,DEVELOPMENT
1,China,2020-11-10 00:00:00,H.E. Xi Jinping,Carrying Forward the Shanghai Spirit and Deepe...,"Your Excellency President Vladimir Putin,\nDea...",President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
2,USA,2017-04-18 00:00:00,Donald Trump,"Remarks by President Trump on Buy American, Hi...",THE PRESIDENT: Thank you very much. (Applaus...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,1,DEVELOPMENT
3,Russia,2017-09-05 00:00:00,Vladimir Putin,Military parade on Red Square,"Citizens of Russia, veterans, comrade soldiers...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,2,NATIONALISM
4,Russia,17-09-2003,Vladimir Putin,Speech at a Meeting on Issues of the Russian M...,"Good afternoon, dear colleagues. First of all,...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,INTERNATIONAL AFFAIRS
...,...,...,...,...,...,...,...,...,...,...
2005,United Kingdom,24 July 2019,Boris Johnson,Boris Johnson's first speech as Prime Minister...,Good afternoon I have just been to see Her Maj...,Prime Minister,Boris Johnson,https://www.gov.uk/government/speeches/boris-j...,1,DEVELOPMENT
2006,Russia,2009-09-02 00:00:00,Dmitry Medvedev,Opening Remarks at Meeting on Economic Issues,"Colleagues,\n\nToday we are meeting to discuss...",President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,3,DEVELOPMENT
2007,China,2018-07-20 00:00:00,H.E. Xi Jinping,Full text of Chinese President Xi's signed art...,I will pay a state visit to the United Arab Em...,President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
2008,Russia,2007-09-05 00:00:00,Vladimir Putin,Speech at the Military Parade Celebrating the ...,"Citizens of Russia,\n\nVeterans of the Great P...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,NATIONALISM


In [274]:
# wf = wf.head(739)

In [275]:
wf.iloc[76]['Text_of_Speech']

'David Chipman spent 25 years in distinguished service to our country as an ATF agent. He’s a gun owner himself, and someone who has the backing of law enforcement groups. And, he’s spent most of last decade as a leading voice for commonsense gun violence prevention legislation that will save lives. He would have been an exemplary Director of the ATF and would have redoubled its efforts to crack down on illegal firearms traffickers and help keep our communities safe from gun violence.\n\nUnfortunately, Republicans in Congress have made clear that they intend to use gun crime as a political talking point instead of taking serious steps to address it. That’s why they’ve moved in lockstep to block David Chipman’s confirmation, and it’s why they side with gun manufacturers over the overwhelming majority of the American people in opposing commonsense measures like universal background checks.\n\nThey even unanimously opposed the Rescue Plan, which has given cities and states $350 billion th

In [276]:
## one thing we can do is to use the model from the presidential classifier and then, if a given speech is classified
## as angry or some other negative mood, and it includes the word "US, NATO", or something related to the west, we classify it as adversarial

In [277]:
#removing the punctuation
def remove_punctuation(speech):
    translator = str.maketrans("", "", string.punctuation)
    return speech.translate(translator)

In [278]:
#removing the stop words (commonly used words that do not add much meaning to a sentence)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    #removes stop words
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to /home/yash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [279]:
#drop rows without string in speech
for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['Text_of_Speech'], str)):
                wf = wf.drop(i)

In [280]:
#counting the number of unique words in each speech


def count_unique_word(wf_training):
    count = Counter()
    for i in range(len(wf_training)):
        for word in wf_training.iloc[i]['Text_of_Speech'].split():
            count[word] += 1
    return count

counter = count_unique_word(wf)

In [281]:
counter

Counter({'the': 134604,
         'and': 96168,
         'to': 85823,
         'of': 78755,
         'in': 46581,
         'a': 42816,
         'that': 33939,
         'is': 28478,
         'we': 26510,
         'for': 26221,
         'our': 23604,
         'I': 22167,
         'have': 17936,
         'are': 17502,
         'this': 16576,
         'on': 16571,
         'will': 15741,
         'with': 15699,
         'be': 13696,
         'And': 13589,
         'as': 13231,
         'We': 11975,
         'you': 11408,
         'it': 11100,
         'all': 10816,
         'not': 10565,
         'has': 8993,
         'their': 8943,
         'The': 8942,
         'more': 8673,
         'by': 8321,
         '—': 8182,
         'from': 7947,
         'people': 7455,
         'at': 7335,
         'an': 7097,
         'who': 7002,
         'they': 6790,
         'can': 6710,
         'new': 5860,
         'was': 5826,
         'but': 5470,
         'China': 5420,
         'been': 5363,
        

In [282]:
num_unique_words = len(counter)

In [283]:
print(len(wf))

2010


In [284]:
wf_training = wf[:int(len(wf) * 0.8)]
wf_testing = wf[int(len(wf) * 0.8):]

In [285]:
wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing

/tmp/ipykernel_1108/3894747268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_testing.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-05-26 00:00:00,Joe Biden,Statement by President Joe Biden on the Crisis...,I am deeply concerned by the escalating violen...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,3,INTERNATIONAL AFFAIRS
1,USA,2021-03-08 00:00:00,Joe Biden,Statement by President Biden on the Introducti...,I applaud the House of Representatives for int...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,1,OTHERS
2,USA,15/03/2014,Barack Obama,\nWeekly Address: Rewarding Hard Work by Stren...,"Hi, everybody. In this year of action, I’m d...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,DEVELOPMENT
3,Russia,31-01-2001,Vladimir Putin,Opening Remarks at a Meeting with New Federati...,Good afternoon.\n\nI am very glad of the oppor...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
4,USA,22/10/2015,Barack Obama,\nRemarks by the President at Veto Signing of ...,"As President and Commander-in-Chief, my first ...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,3,OTHERS
...,...,...,...,...,...,...,...,...,...,...
397,United Kingdom,24 July 2019,Boris Johnson,Boris Johnson's first speech as Prime Minister...,Good afternoon I have just been to see Her Maj...,Prime Minister,Boris Johnson,https://www.gov.uk/government/speeches/boris-j...,1,DEVELOPMENT
398,Russia,2009-09-02 00:00:00,Dmitry Medvedev,Opening Remarks at Meeting on Economic Issues,"Colleagues,\n\nToday we are meeting to discuss...",President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,3,DEVELOPMENT
399,China,2018-07-20 00:00:00,H.E. Xi Jinping,Full text of Chinese President Xi's signed art...,I will pay a state visit to the United Arab Em...,President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
400,Russia,2007-09-05 00:00:00,Vladimir Putin,Speech at the Military Parade Celebrating the ...,"Citizens of Russia,\n\nVeterans of the Great P...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,NATIONALISM


In [286]:
wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training

/tmp/ipykernel_1108/1694703126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_training.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,27/03/2015,Barack Obama,\nRemarks by the President in Meeting with Mem...,I’m happy to have an opportunity to meet with...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,DEVELOPMENT
1,China,2020-11-10 00:00:00,H.E. Xi Jinping,Carrying Forward the Shanghai Spirit and Deepe...,"Your Excellency President Vladimir Putin,\nDea...",President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,0,INTERNATIONAL AFFAIRS
2,USA,2017-04-18 00:00:00,Donald Trump,"Remarks by President Trump on Buy American, Hi...",THE PRESIDENT: Thank you very much. (Applaus...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,1,DEVELOPMENT
3,Russia,2017-09-05 00:00:00,Vladimir Putin,Military parade on Red Square,"Citizens of Russia, veterans, comrade soldiers...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,2,NATIONALISM
4,Russia,17-09-2003,Vladimir Putin,Speech at a Meeting on Issues of the Russian M...,"Good afternoon, dear colleagues. First of all,...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,INTERNATIONAL AFFAIRS
...,...,...,...,...,...,...,...,...,...,...
1603,United Kingdom,25 October 2010,David Cameron,"PM's speech on creating a ""new economic dynamism""",Transcript of speech given by the Prime Minist...,Prime Minister,David Cameron,https://www.gov.uk/government/speeches/pms-spe...,1,DEVELOPMENT
1604,Russia,25-01-2011,Dmitry Medvedev,”This is an act of terror. This is grief. This...,Considering the location and other indirect in...,President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,3,EXTREMISM
1605,United Kingdom,24 September 2019,Boris Johnson,PM speech to the UN General Assembly: 24 Septe...,"Mr President, Your Excellencies, Ladies and Ge...",Prime Minister,Boris Johnson,https://www.gov.uk/government/speeches/pm-spee...,3,DEVELOPMENT
1606,United Kingdom,16 October 2016,Theresa May,Anti-slavery service: Prime Minister's speech,More than 200 years ago William Wilberforce le...,Prime Minister,Theresa May,https://www.gov.uk/government/speeches/anti-sl...,1,DEVELOPMENT


In [287]:
training_sentences = wf_training.Text_of_Speech.to_numpy()
#training labels
training_labels = wf_training.Emotion.to_numpy()

validation_sentences = wf_testing.Text_of_Speech.to_numpy()
#training labels
validation_labels = wf_testing.Emotion.to_numpy()

In [288]:
training_sentences.shape, validation_sentences.shape

((1608,), (402,))

In [289]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)

In [290]:
word_index = tokenizer.word_index

In [291]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

In [292]:
#padding sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

#max number of words per sequence
max_length = 1000

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape

((1608, 1000), (402, 1000))

In [293]:
reverse_word_index_dictionary = dict([(idx, word) for (word, idx) in word_index.items()])

In [294]:
def decode(sequence):
    #takes the sequence and decodes it according to the reverse dictionary
    return " ".join([reverse_word_index_dictionary.get(idx, "?") for idx in sequence])

In [295]:
#LSTM Model
import tensorflow.keras as keras


model = keras.models.Sequential()

#input layer
#embedding layer converts the words into vectors of fixed size
model.add(layers.Embedding(num_unique_words, 32))
#dropout rate drops out a certain percentage of input data so that the
#model learns more robust ways of learning and does not overfit
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [296]:
loss = 'sparse_categorical_crossentropy'
optim = keras.optimizers.Adam(learning_rate=0.001) #from 0.001
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [297]:
print(training_padded.dtype, training_padded.shape)

int32 (1608, 1000)


In [298]:

training_labels = np.array(training_labels).astype(np.int64)
validation_labels = np.array(validation_labels).astype(np.int64)

In [299]:
print(training_padded.dtype, training_padded.shape)
print(training_labels.dtype, training_labels.shape)

int32 (1608, 1000)
int64 (1608,)


In [300]:
print(training_padded)

[[ 196 1033    3 ...    0    0    0]
 [  63 2391  132 ...  977 1459    8]
 [   1  132   69 ...    3  777   84]
 ...
 [ 440  132   63 ...   15   46 1621]
 [  27   72 1503 ...    0    0    0]
 [ 440 2832  332 ...    0    0    0]]


In [301]:
##making sure that overrepresentation of a class does not affect the model
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(training_labels),
    y=training_labels
)
class_weights = dict(enumerate(class_weights))

In [ ]:
model.fit(training_padded, training_labels, epochs=20, validation_data=(validation_padded, validation_labels), class_weight=class_weights, verbose=2)

Epoch 1/20


# Changing to a Binary Classification Model